### We will collect text data using Twitter API.

In [4]:
!pip install tweepy

In [5]:
!pip install openpyxl

In [6]:
import numpy as np
import pandas as pd
import tweepy
import json
from tweepy import OAuthHandler

# Initializing Twitter API

In [7]:
consumer_key = "DreBJqr1OxQZGAuMJIS3kaeTL"
consumer_secret = "5dMdE5ptm468W5o5vQf48AMShNnYGQDvtRkuhcIyycz4wBGKIj"
access_token = "1322035865615556609-z6ZRy7ZJFgNNNleEE4u6zRZ7pNTvTm"
access_token_secret = "Ty0OUxG0ng1YvqihRiDFmhAvOH7j2QhNBbUzWbBi07CH9"

In [8]:
auth = tweepy.OAuthHandler( consumer_key , consumer_secret )
auth.set_access_token( access_token , access_token_secret )
api = tweepy.API(auth)

In [9]:
query = "Narendra Modi Prime Minister"

In [10]:
Tweets = api.search( query , count=10 , lang='en' , exclude='retweets' , tweet_mode='extended' )
#tweet_mode='extended'
# tweet_mode='compat'

# for tweet in Tweets:
#     print(tweet,"\n")
#     print(".....................................\n")

## Creating Dataframe by fetching data using tweepy

In [11]:
df = pd.DataFrame(columns = ['Tweets' , 'User' , 'User_statuses_count' , 
                            'user_followers' , 'User_location' , 'User_verified' ,
                            'fav_count' , 'rt_count' , 'tweet_date'] )
print(df)
print(df.shape)

Empty DataFrame
Columns: [Tweets, User, User_statuses_count, user_followers, User_location, User_verified, fav_count, rt_count, tweet_date]
Index: []
(0, 9)


In [12]:
def stream(data, file_name):
    i = 0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'User_location'] = tweet.user.location
        df.loc[i, 'User_verified'] = tweet.user.verified
        df.loc[i, 'fav_count'] = tweet.favorite_count
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc[i, 'tweet_date'] = tweet.created_at
        df.to_excel('{}.xlsx'.format(file_name))
        i = i+1
        if i == 1000:
            break
        else:
            pass

In [13]:
stream(data=[query] , file_name='my_tweets')

In [14]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date
0,"Shri. Narendra Modi, Honourable Prime Minister...",Suman Sarkar,53,65,London,False,1,0,2020-11-20 14:17:26
1,RT @Warrior35685326: #SaveChildrenOfIIOJK \nTh...,Engrt Ansar Awan,5309,346,Talagang,False,0,2,2020-11-20 14:16:47
2,RT @EconomicTimes: UP CM Yogi Adityanath said ...,Ramky,634,8,,False,0,1,2020-11-20 14:16:10
3,"RT @iampayalghosh: Shri. Narendra Modi, Honour...",Vinay 🇮🇳,169808,1778,India,False,0,20,2020-11-20 14:16:00
4,RT @abhiram_mahakul: PM holds India – Luxembou...,DaalBatiChurmaa,13667,51,,False,0,80,2020-11-20 14:15:41


In [15]:
!pip install textblob

## Cleaning Tweets and analyzing using TextBlob

In [16]:
from textblob import TextBlob

In [17]:
import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [18]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

In [19]:
df['clean_tweet'] = df['Tweets'].apply(lambda x : clean_tweet(x))

In [20]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet
0,"Shri. Narendra Modi, Honourable Prime Minister...",Suman Sarkar,53,65,London,False,1,0,2020-11-20 14:17:26,Shri Narendra Modi Honourable Prime Minister O...
1,RT @Warrior35685326: #SaveChildrenOfIIOJK \nTh...,Engrt Ansar Awan,5309,346,Talagang,False,0,2,2020-11-20 14:16:47,RT SaveChildrenOfIIOJK This ensures that the l...
2,RT @EconomicTimes: UP CM Yogi Adityanath said ...,Ramky,634,8,,False,0,1,2020-11-20 14:16:10,RT UP CM Yogi Adityanath said the National Edu...
3,"RT @iampayalghosh: Shri. Narendra Modi, Honour...",Vinay 🇮🇳,169808,1778,India,False,0,20,2020-11-20 14:16:00,RT Shri Narendra Modi Honourable Prime Ministe...
4,RT @abhiram_mahakul: PM holds India – Luxembou...,DaalBatiChurmaa,13667,51,,False,0,80,2020-11-20 14:15:41,RT mahakul PM holds India Luxembourg Virtual S...


In [21]:
df['Sentiment'] = df['clean_tweet'].apply(lambda x : analyze_sentiment(x) )
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet,Sentiment
0,"Shri. Narendra Modi, Honourable Prime Minister...",Suman Sarkar,53,65,London,False,1,0,2020-11-20 14:17:26,Shri Narendra Modi Honourable Prime Minister O...,Positive
1,RT @Warrior35685326: #SaveChildrenOfIIOJK \nTh...,Engrt Ansar Awan,5309,346,Talagang,False,0,2,2020-11-20 14:16:47,RT SaveChildrenOfIIOJK This ensures that the l...,Neutral
2,RT @EconomicTimes: UP CM Yogi Adityanath said ...,Ramky,634,8,,False,0,1,2020-11-20 14:16:10,RT UP CM Yogi Adityanath said the National Edu...,Neutral
3,"RT @iampayalghosh: Shri. Narendra Modi, Honour...",Vinay 🇮🇳,169808,1778,India,False,0,20,2020-11-20 14:16:00,RT Shri Narendra Modi Honourable Prime Ministe...,Neutral
4,RT @abhiram_mahakul: PM holds India – Luxembou...,DaalBatiChurmaa,13667,51,,False,0,80,2020-11-20 14:15:41,RT mahakul PM holds India Luxembourg Virtual S...,Positive


In [25]:
df[df['User_verified'] == True].count()

Tweets                 48
User                   48
User_statuses_count    48
user_followers         48
User_location          48
User_verified          48
fav_count              48
rt_count               48
tweet_date             48
clean_tweet            48
Sentiment              48
dtype: int64

In [24]:
n = 1
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @Warrior35685326: #SaveChildrenOfIIOJK 
This ensures that the legislature of India, armed forces and Prime Minister Narendra Modi are aw…

Clean tweet:
 RT SaveChildrenOfIIOJK This ensures that the legislature of India armed forces and Prime Minister Narendra Modi are aw

Sentiment of the tweet:
 Neutral


In [26]:
n = 20
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 PM’s Greetings on Chhath
https://t.co/NTEbTP3CdQ via NaMo App https://t.co/UuF8VKnYtp

Clean tweet:
 PM s Greetings on Chhath via NaMo App

Sentiment of the tweet:
 Neutral


In [27]:
n = 203
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @RadharamnDas: India got freedom because of Netaji Subhas Chandra Bose. 

Vested interest banished him from our history books hoping tha…

Clean tweet:
 RT India got freedom because of Netaji Subhas Chandra Bose Vested interest banished him from our history books hoping tha

Sentiment of the tweet:
 Neutral


In [28]:
df[df.Sentiment == 'Positive'].shape[0]

403

In [29]:
df[df.Sentiment == 'Neutral'].shape[0]

591

In [30]:
df[df.Sentiment == 'Negative'].shape[0]

6